# 2. Modeling Probability of Default

    Modeling the probability that a loan defaults is critical to our investment strategy.

As shown in our careful exploration of 2013 and 2014 data, we selected the following predictors to model the probability of a loan defaulting.

- `funded_amnt`: funded amount
- `home_ownership` : home ownership
- `int_rate` : interest rate
- `purpose` : purpose of the loan
- `annual_inc`
- `verification_status`
- `dti`
- `revol_util`
- `grade`
- `term` 

Predicted variable:

Whether a loan_status was paid or fully paif




In [ ]:

    cols = ['funded_amnt', 'emp_length', 'home_ownership', 'int_rate', 'purpose', 'total_pymnt',
        'annual_inc', 'verification_status', 'dti', 'loan_status', 'revol_util', 'grade', 
            'term', 'term_adj', 'zip_code', 'delinq_2yrs']
    # Cols to one hot encode
    cols_encode = ['emp_length', 'home_ownership', 'verification_status', 'grade', 'purpose']
    # Cols to normalize
    cols_normalize = ['funded_amnt', 'annual_inc', 'dti']
    
    cols_to_drop_training = ['loan_status', 'paid', 'amnt', 'total_pymnt', 'term_adj', 'zip_code']
